In [2]:
%matplotlib notebook
import numpy as np
from numpy.fft import fft2, ifft2, fftshift
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import scipy
from scipy.ndimage import gaussian_filter1d as gf1d
from scipy.ndimage import gaussian_filter as gf
from scipy.ndimage import uniform_filter as uf

import sys
sys.path.append("../kai_colloids/PyDDM") #must point to the PyDDM folder
import ddm_analysis_and_fitting as ddm

import tiff_file
import ddm_clean 

import io 
import sys
import csv
import os
import glob #glob is helpful for searching for filenames or directories
import pickle #for saving data

In [3]:
def last_3chars(x):
    return((x.split('\\')[-1])[-7:-4])

In [3]:
###################################################################################
# Specify where the data is and file name for image (or image sequence)           #
###################################################################################

directory = "Z"
exp = "10-15-22_s1_theBigOne2"
data_dir = directory+":\\Gregor L\\__Kai Colloids\\"+exp+"\\all tiff files\\" 
data_dir2 = directory+":\\Gregor L\\__Kai Colloids\\"+exp+"\\reordered tiffs\\" #bottom_row_t01\\
plot_saveto= directory+":\\Gregor L\\__Kai Colloids\\SIA\\Data\\"+exp+"\\"

files = glob.glob(data_dir+"*_t*")
#files = sorted(files, key = last_3chars)
print("found %i files" % len(files))
print(int((len(files))/3))
for i,f in enumerate(files): print (' %i \t %s' % (i, f.split('\\')[-1]))

found 36 files
12
 0 	 row1_t01.tif
 1 	 row1_t02.tif
 2 	 row1_t03.tif
 3 	 row1_t04.tif
 4 	 row1_t05.tif
 5 	 row1_t06.tif
 6 	 row1_t07.tif
 7 	 row1_t08.tif
 8 	 row1_t09.tif
 9 	 row1_t10.tif
 10 	 row1_t11.tif
 11 	 row1_t12.tif
 12 	 row2_t01.tif
 13 	 row2_t02.tif
 14 	 row2_t03.tif
 15 	 row2_t04.tif
 16 	 row2_t05.tif
 17 	 row2_t06.tif
 18 	 row2_t07.tif
 19 	 row2_t08.tif
 20 	 row2_t09.tif
 21 	 row2_t10.tif
 22 	 row2_t11.tif
 23 	 row2_t12.tif
 24 	 row3_t01.tif
 25 	 row3_t02.tif
 26 	 row3_t03.tif
 27 	 row3_t04.tif
 28 	 row3_t05.tif
 29 	 row3_t06.tif
 30 	 row3_t07.tif
 31 	 row3_t08.tif
 32 	 row3_t09.tif
 33 	 row3_t10.tif
 34 	 row3_t11.tif
 35 	 row3_t12.tif


In [4]:
###### # specify each condition (frame_names) and the time points of data collection (time_array) #
############################################################################################
frame_names = ["1-3 kA-WT", "WT (no kA)", "EA KaiC", "AE KaiC"]
time_array =  [1, 3.83, 6.83, 10.17, 13.08, 18, 21.92, 25.25, 28.67, 45.92, 49.33, 69.42]
#[0.5, 4.5, 8, 12, 20, 24]
#[0.5, 3.5, 6.5, 9.3, 12.1, 15, 18, 21, 24.5, 27.3, 40.5, 44.3]  
#[0.5, 3.2, 17.75, 20, 22, 24.25, 26, 41.75, 44]
#[0.85, 3.5, 18, 20.4, 22.25, 24.5, 26.2, 41.75, 44.2]
#s1 [0.67, 3.67, 6.67, 10, 12.83, 17.67, 21.67, 24.92, 28.17, 45.67, 49.08, 69.25]
#s2 [1, 3.83, 6.83, 10.17, 13.08, 18, 21.92, 25.25, 28.67, 45.92, 49.33, 69.42]
fig_size = 10,10/1.618
font_size = 16
dpi_num = 800
pixel_size = 0.364 # 4*0.091 = 0.364

print((files[0]))
eg_im= tiff_file.imread(files[0])
print(eg_im.shape)
frames_per_tiff = int(eg_im.shape[0])
print(frames_per_tiff)
arr_length = int((len(files))/3)
print(arr_length)

Z:\Gregor L\__Kai Colloids\9-5-22_s2_theBigOne\all tiff files\row1_t01.tif


C:\Users\gleech\Documents\GitHub\kai codes\kai_colloids\tiff_file.py:724: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.vstack((p.asarray() if p else nopage)


(5, 1440, 1920)
5
12


In [27]:
###################################################################################################
# makes new tiff files, each new tiff has all consecutive time points for each specific condition #
###################################################################################################
arr_length = int((len(files))/3)

for j in range(frames_per_tiff-1):
    frame_num = str(j+1)

    for i in range(arr_length):    
        row1_im = tiff_file.imread(files[i],key=[j])
        row2_im = tiff_file.imread(files[i+arr_length],key=[j])
        row3_im = tiff_file.imread(files[i+(arr_length*2)],key=[j])
        #print(br_frame1_im.shape[0])
        if i==0:
            row1 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row2 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
            row3 = np.zeros((arr_length, row1_im.shape[0], row1_im.shape[1]), dtype=np.uint16)
        row1[i] = row1_im
        row2[i] = row2_im
        row3[i] = row3_im

    tiff_file.imsave(data_dir2 +'row1_'+frame_names[j]+'.tif', row1)
    tiff_file.imsave(data_dir2 +'row2_'+frame_names[j]+'.tif', row2)
    tiff_file.imsave(data_dir2 +'row3_'+frame_names[j]+'.tif', row3)
    print(row1.shape)

found 72 files
 0 	 20x_row1_g01
 1 	 20x_row1_g02
 2 	 20x_row1_g03
 3 	 20x_row1_g04
 4 	 20x_row1_g05
 5 	 20x_row1_g06
 6 	 20x_row1_g07
 7 	 20x_row1_g08
 8 	 20x_row1_g09
 9 	 20x_row1_g10
 10 	 20x_row1_g11
 11 	 20x_row1_g12
 12 	 20x_row2_g01
 13 	 20x_row2_g02
 14 	 20x_row2_g03
 15 	 20x_row2_g04
 16 	 20x_row2_g05
 17 	 20x_row2_g06
 18 	 20x_row2_g07
 19 	 20x_row2_g08
 20 	 20x_row2_g09
 21 	 20x_row2_g10
 22 	 20x_row2_g11
 23 	 20x_row2_g12
 24 	 20x_row3_g01
 25 	 20x_row3_g02
 26 	 20x_row3_g03
 27 	 20x_row3_g04
 28 	 20x_row3_g05
 29 	 20x_row3_g06
 30 	 20x_row3_g07
 31 	 20x_row3_g08
 32 	 20x_row3_g09
 33 	 20x_row3_g10
 34 	 20x_row3_g11
 35 	 20x_row3_g12
 36 	 row1_t01
 37 	 row1_t02
 38 	 row1_t03
 39 	 row1_t04
 40 	 row1_t05
 41 	 row1_t06
 42 	 row1_t07
 43 	 row1_t08
 44 	 row1_t09
 45 	 row1_t10
 46 	 row1_t11
 47 	 row1_t12
 48 	 row2_t01
 49 	 row2_t02
 50 	 row2_t03
 51 	 row2_t04
 52 	 row2_t05
 53 	 row2_t06
 54 	 row2_t07
 55 	 row2_t08
 56 	 row2_

In [28]:
#####################################################
# **optional** preview of tiff files to be analyzed #
#####################################################
tiff_name = 'row3_'+frame_names[0]
tiff_to_show = tiff_file.imread(data_dir2 +tiff_name+'.tif')

fig = plt.figure(figsize=(10,10))    
for i in range (arr_length):
    ax = plt.subplot(4,4,i+1) ##change according to number of frames (6 --> 3,3; 12 --> 4,4)
    ax.set_title("~" + str(time_array[i]) + " hrs", fontsize=10)
    ax.matshow(tiff_to_show[i], cmap = 'gray') #fig.set_cmap('hot') 
    plt.axis('off')
    plt.tight_layout(pad=.25)
fig.savefig(data_dir2+tiff_name+".jpg", dpi=(dpi_num))

TypeError: len() takes exactly one argument (0 given)